In [63]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
from ollama import Client
client = Client(host="http://localhost:11434")
model_name = "qwen2.5:7b"


In [65]:
from src.agentic.tool import ToolAgent
from functions import summation, multiplication

In [66]:
agent = ToolAgent(client=client, toolkit=[summation, multiplication], model="qwen2.5:7b")

In [67]:
response = agent.start(message="Can you multiple 25 by 12 and sum 3 with 6 and search what is an LLM on google.com?")

In [68]:
from src.agentic.planning import PlanningAgent

In [69]:
pagent = PlanningAgent(client=client, toolkit=[summation, multiplication], model="qwen2.5:7b")

In [70]:
result = pagent.start(message="Can you multiple 25 by 12 and sum the result by 3 and multiply by 12 and then sum by 354?")

In [71]:
print(result)

The final result of the operations is 3990. 

<response>The result of multiplying 25 by 12, summing it by 3, multiplying by 12 and then summing by 354 is 3990.</response>


In [72]:
from src.tool import Function, Tool, ToolKit
from typing import List

In [73]:
def sum2(a: float, b: float) -> float:
    return a + b

def sub(a: float, b: float) -> float:
    return a - b

def list_mul(a: int, b: List = [1,2,3]) -> List:
    return a * b

In [74]:
sum2_func = Function(sum2)
sub_func = Function(sub)

In [75]:
sub_func

Function(name=sub, signature=(a: float, b: float) -> float, doc=None)

In [76]:
sum2_func.run({'a': 2, 'b': 3})

5.0

In [77]:
sub_func.run({'a': 2, 'b': 3})

-1.0

In [78]:
sum2_tool = Tool(func=sum2)

In [79]:
sum2_tool.schema

{'name': 'sum2',
 'doc': None,
 'parameters': {'properties': {'a': {'type': 'float'},
   'b': {'type': 'float'}}}}

In [80]:
sum2_tool.run({'a': 2, 'b': 3})

5.0

In [81]:
assert sum2_tool.run({'a': 2, 'b': 3}) == sum2_func.run({'a': 2, 'b': 3})

In [82]:
sum2_tool.run({'a': 2, 'b': 3})

5.0

In [83]:
tk = ToolKit(tools=[sum2, sub, list_mul])

In [84]:
tk.tools

[Tool(name=sum2, signature=(a: float, b: float) -> float, doc=None),
 Tool(name=sub, signature=(a: float, b: float) -> float, doc=None),
 Tool(name=list_mul, signature=(a: int, b: List = [1, 2, 3]) -> List, doc=None)]

In [85]:
assert tk.tools[0].run({'a': 2, 'b': 3}) == sum2_tool.run({'a': 2, 'b': 3}) == sum2_func.run({'a': 2, 'b': 3})

In [86]:
assert tk.get_tool_by_name('sum2').run({'a': 2, 'b': 3}) == sum2_tool.run({'a': 2, 'b': 3}) == sum2_func.run({'a': 2, 'b': 3})

In [87]:
from src.tool import ToolCallProcessor
# Example usage
processor = ToolCallProcessor("""
<tool_call>
{"name": "summation", "arguments": {"x": 3, "y": 6}, "id": 2}
</tool_call>
<tool_call>
{"name": "search", "arguments": {"query": "what is an LLM on google.com"}, "id": 3}
</tool_call>
""")


In [88]:
for call in processor.calls:
    print(call)